# 🚀 Master Notebook – Pipeline Spectroscopie DR5

Ce notebook orchestre le **workflow complet** pour entraîner un classifieur à partir des spectres LAMOST DR5 :  
**sélection d’un lot → génération/enrichissement du catalogue → prétraitement + features → entraînement → journaux & artefacts.**

**Sommaire rapide**
- [🧪 Étape 0 : SETUP & IMPORTS](#etape-0)
- [▶️ Lancer une session complète](#run-full)
- [1) Téléchargement des spectres](#step-1-download)
- [2) Sélection du lot de spectres](#step-2-select)
- [3) Catalogue : génération & enrichissement Gaia](#step-3-catalog)
- [3bis) Traitement & extraction des features](#step-3bis-features)
- [4) Entraînement du modèle](#step-4-train)

> Orchestrateur utilisé : **`MasterPipeline`**  
> Méthodes clés : `select_batch`, `generate_and_enrich_catalog`, `process_data`, `run_training_session`, `run_full_pipeline`, `interactive_training_runner`.

<a id="etape-0"></a>

#

## 🧪 Étape 0 : SETUP & IMPORTS

Initialise l’environnement, crée/valide l’arborescence des répertoires
(`RAW_DATA_DIR`, `CATALOG_DIR`, `PROCESSED_DIR`, `MODELS_DIR`, `REPORTS_DIR`)
et instancie **`MasterPipeline`**.

**Attendu après exécution :**
- un objet `pipeline` prêt à l’emploi,
- messages sur la racine du projet et, si configuré, tentative de connexion à **Gaia**.

> ℹ️ **UI interactive d’entraînement** : disponible avec `pipeline.interactive_training_runner()`.

In [1]:
from utils import setup_project_env, load_env_vars
from pipeline.master import MasterPipeline
from astroquery.gaia import Gaia
from pipeline.classifier import SpectralClassifier

# Initialisation automatique de l'environnement et des chemins
paths = setup_project_env()

# Chargement des credentials Gaia depuis .env
env_vars = load_env_vars()

try:
    print("Tentative de connexion à l'archive Gaia...")
    Gaia.login(user=env_vars.get("GAIA_USER"), password=env_vars.get("GAIA_PASS"))
    print("Connexion à Gaia réussie.")
except Exception as e:
    print(f"AVERTISSEMENT : Échec de la connexion à Gaia ({e}). Le mode 'bulk' pourrait échouer.")

# Instanciation du pipeline maître
pipeline = MasterPipeline(
    raw_data_dir=paths["RAW_DATA_DIR"],
    catalog_dir=paths["CATALOG_DIR"],
    processed_dir=paths["PROCESSED_DIR"],
    models_dir=paths["MODELS_DIR"],
    reports_dir=paths["REPORTS_DIR"],
)

print("\nSetup terminé. Tu es prêt à lancer ton pipeline.")

[INFO] Racine du projet détectée : C:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git
[INFO] Dossier 'src' ajouté au sys.path.
[INFO] Variables d'environnement chargées depuis C:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git\.env
Tentative de connexion à l'archive Gaia...
INFO: Login to gaia TAP server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
INFO: Login to gaia data server [astroquery.gaia.core]
INFO: OK [astroquery.utils.tap.core]
Connexion à Gaia réussie.

Setup terminé. Tu es prêt à lancer ton pipeline.


#

---

<a id="run-full"></a>
## ▶️ Lancer une session complète

Lance **tout le pipeline A→Z** :
`select_batch → generate_and_enrich_catalog → process_data → run_training_session`.

> 💡 **Paramètres conseillés** au début : `batch_size=200–500`, `n_estimators=200–400` (RF/XGB).  
> Active `enrich_gaia=True` lorsque la connectivité est stable.

In [ ]:
pipeline.run_full_pipeline(
    batch_size=500,                 # taille du lot
    model_type="RandomForest",      # "RandomForest" ou "XGBoost"
    n_estimators=100,               # arbres du modèle final
    prediction_target="main_class", # ex.: "main_class", "sub_class_top25", "sub_class_bins"
    save_and_log=True,              # sauvegarde modèle + rapport JSON
    enrich_gaia=False,              # True pour activer Gaia
    # ...kwargs Gaia si enrich_gaia=True
)

#

---

<a id="step-1-download"></a>
## 1) Téléchargement des spectres

Utilisation du script **`dr5_downloader.py`** encapsulé.  
Cette étape est externalisée dans **[01_download_spectra.ipynb](./01_download_spectra.ipynb)** (à exécuter au besoin).

> ⚠️ **Quota / temps** : selon le volume demandé, le téléchargement peut être long.

#

<a id="step-2-select"></a>
## 2) Sélection du lot de spectres

Choisit un **nouveau lot** de fichiers `.fits.gz` à traiter sans réutiliser de spectres déjà journalisés.

- `batch_size` : nombre de spectres,
- `strategy` : ex. `"random"`.

Le sélectionneur s’appuie sur **DatasetBuilder** pour garantir l’unicité des échantillons.


In [2]:
pipeline.select_batch(batch_size=3000, strategy="random")


=== ÉTAPE 1 : SÉLECTION D'UN NOUVEAU LOT ===
--- Constitution d'un nouveau lot d'entraînement ---
  > 43392 spectres trouvés dans 'C:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git\data\raw'
  > 13000 spectres déjà utilisés dans le journal.
  > 30392 spectres **nouveaux** disponibles.
  > Sélection aléatoire de 3000 spectres.


['M5901/spec-55859-M5901_sp01-195.fits.gz',
 'M6201/spec-55862-M6201_sp11-115.fits.gz',
 'GAC_122N29_B1/spec-55874-GAC_122N29_B1_sp01-066.fits.gz',
 'GAC_060N28_B1/spec-55863-GAC_060N28_B1_sp16-109.fits.gz',
 'GAC_105N29_B1/spec-55863-GAC_105N29_B1_sp13-014.fits.gz',
 'GAC_060N28_B1/spec-55863-GAC_060N28_B1_sp08-119.fits.gz',
 'M6201/spec-55862-M6201_sp06-147.fits.gz',
 'F5907/spec-55859-F5907_sp10-064.fits.gz',
 'B6301/spec-55863-B6301_sp09-034.fits.gz',
 'M6201/spec-55862-M6201_sp13-011.fits.gz',
 'GAC_105N29_B1/spec-55863-GAC_105N29_B1_sp07-163.fits.gz',
 'M6201/spec-55862-M6201_sp05-081.fits.gz',
 'M6201/spec-55862-M6201_sp05-003.fits.gz',
 'B7401/spec-55874-B7401_sp07-140.fits.gz',
 'GAC_105N29_B1/spec-55863-GAC_105N29_B1_sp14-019.fits.gz',
 'M6201/spec-55862-M6201_sp06-216.fits.gz',
 'M5901/spec-55859-M5901_sp07-112.fits.gz',
 'B6001/spec-55860-B6001_sp07-049.fits.gz',
 'B6001/spec-55860-B6001_sp01-155.fits.gz',
 'M5901/spec-55859-M5901_sp07-095.fits.gz',
 'M31_011N40_M1/spec-558

#

<a id="step-3-catalog"></a>
## 3) Catalogue : génération & enrichissement Gaia

À partir du lot courant, produit un **catalogue local** (CSV) et peut l’**enrichir via Gaia** (positions, photométrie…).

**Sorties :**
- `master_catalog_temp.csv` (catalogue local) puis `master_catalog_gaia.csv` si enrichi,
- mise à jour de `pipeline.master_catalog_df`.

> ℹ️ **Couplage Gaia** : géré par l’orchestrateur (appairage + stats).  
> ⚠️ **Connexion** : si l’authentification Gaia échoue, relance sans `enrich_gaia` ou vérifie tes identifiants.

In [3]:
pipeline.generate_and_enrich_catalog(
    enrich_gaia=True,
    mode='bulk',
    include_risky=False,   # <-- active radius/mass/age -- beta donc en test
    ruwe_max=1.4           # optionnel: garde aussi les entrées à RUWE élevé - <1.4 est un bon filtre
)


=== ÉTAPE 2 : GÉNÉRATION ET ENRICHISSEMENT DU CATALOGUE ===


Extraction des headers: 100%|██████████| 3000/3000 [00:37<00:00, 79.56fichier/s] 


[OK] Catalogue écrit : C:\Users\alexb\Documents\Google_Cloud\alex_labs_google_sprint\astro_spectro_git\data\catalog\master_catalog_temp.csv  (3000 lignes)
  > Catalogue local de 3000 spectres créé.
  > Tentative de cross-match en mode 'bulk'…
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]
  > Gaia : 2651/3000 objets appariés.


#

<a id="step-3bis-features"></a>
## 3bis) Traitement & extraction des features

Exécute les **prétraitements spectraux** et l’**extraction de features**.  
Un CSV `features_YYYYMMDDTHHMMSSZ.csv` est écrit dans `processed/`.

- Met à jour `pipeline.features_df` (mémoire) & `pipeline.last_features_path` (disque).
- Features = mesures photométriques/astrométriques, indices de raies, résumés de voisinage spectral…

> 💡 Certaines étapes internes reposent sur des détections/associations de raies (Balmer, Ca II H/K, Mg_b, Na_D).

In [4]:
pipeline.process_data()


=== ÉTAPE 3 : TRAITEMENT DES DONNÉES ET EXTRACTION DES FEATURES ===

--- Démarrage du pipeline de traitement pour 3000 spectres ---


Traitement des spectres: 100%|██████████| 3000/3000 [01:32<00:00, 32.28it/s]


  > Création des features de couleur photométrique...

Pipeline de traitement terminé. 3000 spectres traités et enrichis.

  > Dataset de features sauvegardé dans : features_20250820T020828Z.csv


,file_path,feature_Hα_prominence,feature_Hα_fwhm,feature_Hα_eq_width,feature_Hβ_prominence,feature_Hβ_fwhm,feature_Hβ_eq_width,feature_CaIIK_prominence,feature_CaIIK_fwhm,feature_CaIIK_eq_width,...,pmra,pmdec,radial_velocity,distance_gspphot,astrometric_excess_noise,phot_variable_flag,bp_g,g_rp,feature_color_gr,feature_color_ri
0,M5901/spec-55859-M5901_sp01-195.fits.gz,0.897656,0.000000,-25.007062,1.584174,22.257277,14.582983,2.112454,30.612651,34.735588,...,-6.877512,-1.312645,NaN,547.982178,0.000000,NOT_AVAILABLE,1.093296,1.025904,1.60,0.79
1,M6201/spec-55862-M6201_sp11-115.fits.gz,0.394714,0.000000,-10.810985,1.051613,23.918516,12.584038,5.413777,NaN,-7.816648,...,7.078463,-5.884965,NaN,809.528625,0.354053,NOT_AVAILABLE,1.271902,1.089396,1.54,1.01
2,GAC_122N29_B1/spec-55874-GAC_122N29_B1_sp01-06...,0.000000,0.000000,0.000000,0.253845,28.783282,5.656627,1.099345,27.254958,31.305362,...,-2.308271,-1.709090,NaN,957.626526,0.000000,NOT_AVAILABLE,0.782026,0.862169,1.25,0.50
3,GAC_060N28_B1/spec-55863-GAC_060N28_B1_sp16-10...,0.413267,19.550819,5.444872,0.683954,NaN,-0.373323,1.048922,NaN,5.901286,...,-1.211095,-3.415593,-17.403130,977.754578,0.000000,NOT_AVAILABLE,0.589247,0.745384,0.78,0.50
4,GAC_105N29_B1/spec-55863-GAC_105N29_B1_sp13-01...,0.612846,22.183278,7.211732,1.367968,31.939313,-8.790034,3.679062,34.434944,-8.373902,...,0.079957,-2.774574,NaN,1870.028442,0.073624,NOT_AVAILABLE,0.281265,0.443264,0.34,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,M5904/spec-55859-M5904_sp01-016.fits.gz,0.635621,29.834264,-8.458881,0.954457,24.406084,17.638279,7.796431,26.016579,43.957247,...,-0.530957,-2.156844,NaN,624.186096,0.594187,VARIABLE,1.314123,1.182482,1.92,0.98
2996,M5901/spec-55859-M5901_sp14-066.fits.gz,0.210758,28.733648,-3.498011,0.506400,24.435788,-4.875719,1.652133,26.057355,31.594651,...,-2.359372,-4.947338,NaN,1528.496460,0.000000,NOT_AVAILABLE,0.528336,0.659740,0.78,0.29
2997,M6201/spec-55862-M6201_sp03-198.fits.gz,0.363929,24.131413,6.541094,0.831629,29.249404,-9.581387,2.508117,NaN,6.711009,...,-4.742670,-1.496797,NaN,1973.120483,0.000000,NOT_AVAILABLE,0.362835,0.531857,0.53,0.13
2998,GAC_122N29_B1/spec-55874-GAC_122N29_B1_sp14-04...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.267317,26.967935,34.074897,...,-7.056076,-34.412493,24.168312,170.580307,0.000000,NOT_AVAILABLE,1.013433,0.997933,1.44,0.85


#

<a id="step-4-train"></a>
## 4) Entraînement du modèle

Entraîne un **classifieur** (RF/XGBoost) avec **sélection de features** optionnelle (`SelectFromModel`, seuil `"median"` par défaut), puis **évalue** et **journalise**.

- Récap : nb de features conservées, scores, rapports, chemins des artefacts.
- En notebook : UI dédiée via `pipeline.interactive_training_runner()`.

> 💡 **Astuce** : commence avec RF pour un feedback rapide, puis passe à XGB pour gagner en performance.

In [5]:
pipeline.interactive_training_runner()

#
#
#